In [1]:
import pandas as pd
import numpy as np
import re
from random import random
import emoji
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import torch
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from os import listdir
import csv

In [2]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel
from sklearn.preprocessing import MultiLabelBinarizer

I0625 05:24:50.501645 140553350014784 file_utils.py:39] PyTorch version 1.5.0 available.


In [3]:
from simpletransformers.language_modeling import LanguageModelingModel

## Read input data

In [4]:
# df = pd.read_json('./source/train_gold.json', lines=True)
df = pd.read_json('./preprocessed/preprocess_train.json', lines=True)
df_new = df.copy()
df_new.head()

,idx,text,reply,categories
0,0,we can all agree that any song by Niall Horan .,oui oui,[yes]
1,1,Will you be install # ScottyFromMarketing ' ...,,[no]
2,2,Growing up my mum would call me a Nigga despit...,And he joins in ? ? ? Pour some hot grits o...,[smh]
3,3,Rest your head on my chest when the world feel...,😂😂😂😂😂,[wink]
4,4,Imagine Will Hernandez and Wills both doing a ...,,[yes]


In [5]:
categories_type = pd.read_json('./source/categories.json', lines=True)
categories_mapping = {v[0]: k for k, v in categories_type.to_dict('list').items()}

In [6]:
df_dev_ori = pd.read_json('./source/dev_unlabeled.json', lines=True)
df_dev = pd.read_json('./preprocessed/preprocess_dev.json', lines=True)
df_dev_result = df_dev.copy()[['text', 'reply']]
df_dev_result.head()

,text,reply
0,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans
1,After interviewing with a few incredible peopl...,CONGRATS ! ! ! ! !
2,I know GTC festival not happening next month b...,
3,"Lordy , my daughter just said , "" I wonde...",
4,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...


In [7]:
df_test_ori = pd.read_json('./source/test_unlabeled.json', lines=True)
df_test = pd.read_json('./preprocessed/preprocess_test.json', lines=True)
df_test_result = df_test.copy()[['text', 'reply']]
df_test_result.head()

,text,reply
0,@ Youngdeji - I think if uzi and carti dro...,
1,For the third year in a row we are discussing ...,
2,dababy album sounds like it was made for nigga...,That ' s why you buy it .
3,Majority of Indians do not watch any sport oth...,@ ZairaWasimmm got a great story because of t...
4,everybody is just now listen to @ madisonbeer...,


### Use multi-hot encoding and change column name

In [8]:
mlb = MultiLabelBinarizer()
multi_hot = mlb.fit_transform(df_new['categories'].values)
multi_hot_list = [list(_) for _ in multi_hot]

In [9]:
df_new['labels'] = multi_hot_list
df_new = df_new[['text', 'reply', 'labels']]
df_new.columns = ['text_a', 'text_b', 'labels']
df_dev_result.columns = ['text_a', 'text_b']
df_test_result.columns = ['text_a', 'text_b']

# 10 roberta

In [10]:
# Output data to train_file for training LM
df_forLM = df_new.copy()
df_forLM['text'] = ['<s> '] + df_new['text_a'] + [' </s></s> '] + df_new['text_b'] + [' <s>']
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 11):
    train_lm_args = {
        "output_dir": "round1_blend_model/round1_LM/outputs_roberta_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,
    }
    model = LanguageModelingModel("roberta", "roberta-base", args=train_lm_args)
    model.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round1_blend_model/round1_MLR/outputs_roberta_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,
        "evaluate_during_training": False,
        "evaluate_during_training_steps": 2000,
        "evaluate_during_training_verbose": False,
        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 2000,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
        "config": {}
    }
    model = MultiLabelClassificationModel('roberta', 'round1_blend_model/round1_LM/outputs_roberta_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)

I0625 05:25:08.534508 140553350014784 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at cache/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I0625 05:25:08.535140 140553350014784 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at cache/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0625 05:25:09.534883 140553350014784 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at cache/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
I0625 05:25:09.535674 140553350014784 configuration_utils.py:321] Mode

I0625 05:25:24.273219 140553350014784 language_modeling_utils.py:229]  Saving features into cached file cache/roberta_cached_lm_110_training.txt
I0625 05:25:24.305407 140553350014784 language_modeling_model.py:473]  Training started


Running loss: 1.962883

I0625 05:28:31.791547 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_LM/outputs_roberta_LM_training1/checkpoint-829-epoch-1/config.json


Running loss: 1.569290


...

I0625 11:17:32.443406 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/checkpoint-6000-epoch-3/pytorch_model.bin


Running loss: 0.105679

I0625 11:23:04.703508 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/checkpoint-8000/config.json
I0625 11:23:04.943995 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/checkpoint-8000/pytorch_model.bin
I0625 11:23:05.454222 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/checkpoint-8000-epoch-4/config.json


I0625 11:23:05.697383 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/checkpoint-8000-epoch-4/pytorch_model.bin
I0625 11:23:06.286602 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/config.json


I0625 11:23:06.528885 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_roberta_LM_training10/pytorch_model.bin
I0625 11:23:06.569630 140553350014784 classification_model.py:279]  Training of roberta model complete. Saved to round1_blend_model/round1_MLR/outputs_roberta_LM_training10/.


# 5 cased

In [12]:
df_forLM = df_new.copy()
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 6):
    train_lm_args = {
        "output_dir": "round1_blend_model/round1_LM/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,
    }
    model = LanguageModelingModel("bert", "bert-base-cased", args=train_lm_args)
    model.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round1_blend_model/round1_MLR/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,
        "evaluate_during_training": False,
        "evaluate_during_training_steps": 2000,
        "evaluate_during_training_verbose": False,
        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 2000,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
        "config": {}
    }
    model = MultiLabelClassificationModel('bert', 'round1_blend_model/round1_LM/outputs_bert_cased_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)

I0625 14:08:36.781080 140553350014784 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at cache/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I0625 14:08:37.774063 140553350014784 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at cache/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0625 14:08:37.776594 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 5

I0625 14:08:42.832368 140553350014784 language_modeling_utils.py:229]  Saving features into cached file cache/bert_cached_lm_111_training.txt
I0625 14:08:42.862173 140553350014784 language_modeling_model.py:473]  Training started


Running loss: 2.496701

I0625 14:11:08.730382 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_LM/outputs_bert_cased_LM_training1/checkpoint-727-epoch-1/config.json


I0625 14:11:08.954952 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_LM/outputs_bert_cased_LM_training1/checkpoint-727-epoch-1/pytorch_model.bin


...

I0625 16:45:11.519190 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/checkpoint-6000-epoch-3/pytorch_model.bin


Running loss: 0.088493

I0625 16:50:39.792351 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/checkpoint-8000/config.json
I0625 16:50:40.038933 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/checkpoint-8000/pytorch_model.bin
I0625 16:50:40.549069 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/checkpoint-8000-epoch-4/config.json


I0625 16:50:40.795881 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/checkpoint-8000-epoch-4/pytorch_model.bin
I0625 16:50:41.298336 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/config.json


I0625 16:50:41.543707 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/pytorch_model.bin
I0625 16:50:41.556727 140553350014784 classification_model.py:279]  Training of bert model complete. Saved to round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/.


# 5 uncased

In [13]:
df_forLM = df_new.copy()
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 6):
    train_lm_args = {
        "output_dir": "round1_blend_model/round1_LM/outputs_bert_uncased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,
    }
    model = LanguageModelingModel("bert", "bert-base-uncased", args=train_lm_args)
    model.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,
        "evaluate_during_training": False,
        "evaluate_during_training_steps": 2000,
        "evaluate_during_training_verbose": False,
        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 2000,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
        "config": {}
    }
    model = MultiLabelClassificationModel('bert', 'round1_blend_model/round1_LM/outputs_bert_uncased_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)

I0625 16:53:36.430842 140553350014784 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0625 16:53:37.378387 140553350014784 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at cache/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0625 16:53:37.379137 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings

I0625 16:53:47.771228 140553350014784 language_modeling_utils.py:229]  Saving features into cached file cache/bert_cached_lm_111_training.txt


I0625 16:53:47.799728 140553350014784 language_modeling_model.py:473]  Training started


Running loss: 2.934939

I0625 16:56:09.469473 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_LM/outputs_bert_uncased_LM_training1/checkpoint-689-epoch-1/config.json


...

I0625 19:28:21.891513 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/checkpoint-6000-epoch-3/pytorch_model.bin


Running loss: 0.127240

I0625 19:33:47.782159 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/checkpoint-8000/config.json
I0625 19:33:47.992037 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/checkpoint-8000/pytorch_model.bin
I0625 19:33:48.419595 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/checkpoint-8000-epoch-4/config.json


I0625 19:33:48.629297 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/checkpoint-8000-epoch-4/pytorch_model.bin
I0625 19:33:49.057519 140553350014784 configuration_utils.py:144] Configuration saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/config.json


I0625 19:33:49.267709 140553350014784 modeling_utils.py:483] Model weights saved in round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/pytorch_model.bin
I0625 19:33:49.281606 140553350014784 classification_model.py:279]  Training of bert model complete. Saved to round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/.


## Predict results [pair sentance](https://simpletransformers.ai/docs/classification-data-formats/)

### test

In [14]:
to_predict = []
for i in range(len(df_test_result)):
    text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

In [15]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round1_blend_model/round1_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./final_results/test/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0625 20:24:17.790860 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training1/config.json
I0625 20:24:17.794344 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:24:38.277387 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/config.json
I0625 20:24:38.278079 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:24:58.606472 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training3/config.json
I0625 20:24:58.607131 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:25:18.991020 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training4/config.json
I0625 20:25:18.991648 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:25:40.014109 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training5/config.json
I0625 20:25:40.014735 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:26:01.160101 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training6/config.json
I0625 20:26:01.160797 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:26:20.254206 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training7/config.json
I0625 20:26:20.254834 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:26:39.487960 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training8/config.json
I0625 20:26:39.488617 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:26:58.689223 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training9/config.json
I0625 20:26:58.689846 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:27:17.926090 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training10/config.json
I0625 20:27:17.926837 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

In [16]:
to_predict = []
for i in range(len(df_test_result)):
    text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [17]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round1_blend_model/round1_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./final_results/test/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0625 20:28:40.091176 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training1/config.json
I0625 20:28:40.092066 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:28:58.894474 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training2/config.json
I0625 20:28:58.895091 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:29:17.669618 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training3/config.json
I0625 20:29:17.670567 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:29:36.610948 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training4/config.json
I0625 20:29:36.611679 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:29:52.495178 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/config.json
I0625 20:29:52.495778 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:30:08.414053 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training1/config.json
I0625 20:30:08.414676 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:30:27.004646 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training2/config.json
I0625 20:30:27.005337 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:30:45.539209 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training3/config.json
I0625 20:30:45.539832 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:31:04.230490 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training4/config.json
I0625 20:31:04.231113 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:31:22.781734 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/config.json
I0625 20:31:22.782361 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

In [18]:
files = listdir('./final_results/test/')
roberta_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_test_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./final_results/test/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_cased_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncased') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [19]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [23]:
P = 3.5
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [24]:
df_test_ori['categories'] = total_predict
df_test_ori.head()

,idx,text,reply,categories
0,36000,@Youngdeji_ I think if uzi and carti dropping ...,,"[agree, dance, happy_dance, no, please, yes]"
1,36001,For the third year in a row we’re discussing t...,,"[eye_roll, facepalm, no, seriously, sigh, smh]"
2,36002,dababy album sounds like it was made for nigga...,That's why you bought it.,"[agree, omg, seriously, shocked, smh, yes]"
3,36003,Majority of Indians don’t watch any sport othe...,@ZairaWasimmm got a great story because of the...,"[agree, applause, oh_snap, shocked, slow_clap,..."
4,36004,everybody is just now listening to @madisonbee...,,"[facepalm, idk, seriously, shocked, sigh, smh]"


In [25]:
df_test_ori.to_json('./results/eval.json', orient='records', lines=True)

# dev

In [26]:
to_predict = []
for i in range(len(df_dev_result)):
    text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

In [27]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round1_blend_model/round1_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./final_results/dev/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0625 20:34:01.536793 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training1/config.json
I0625 20:34:01.537540 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:34:17.969518 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/config.json
I0625 20:34:17.970340 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:34:20.560892 140553350014784 tokenization_utils.py:929] Model name 'round1_blend_model/round1_MLR/outputs_roberta_LM_training2/' not found in model shortcut name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). Assuming 'round1_blend_model/round1_MLR/outputs_roberta_LM_training2/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0625 20:34:20.561494 140553350014784 tokenization_utils.py:958] Didn't find file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/added_tokens.json. We won't load it.
I0625 20:34:20.561860 140553350014784 tokenization_utils.py:1013] loading file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/vocab.json
I0625 20:34:20.562178 140553350014784 tokenization_utils.py:1013] loading file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/merges.txt
I0625 20:34:20.562509 140553350014784 tokenization_utils.py:1013

I0625 20:34:34.621426 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training3/config.json
I0625 20:34:34.622107 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:34:51.088480 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training4/config.json
I0625 20:34:51.089193 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

...

I0625 20:35:40.752967 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training7/config.json
I0625 20:35:40.753614 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:35:57.272168 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training8/config.json
I0625 20:35:57.272832 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:36:13.788202 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training9/config.json
I0625 20:36:13.788918 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:36:30.441080 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training10/config.json


I0625 20:36:30.441716 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    "30": "LABEL_

In [28]:
to_predict = []
for i in range(len(df_dev_result)):
    text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [29]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round1_blend_model/round1_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./final_results/dev/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0625 20:41:35.630744 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training1/config.json
I0625 20:41:35.631663 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:41:51.503073 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training2/config.json
I0625 20:41:51.503672 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:07.455550 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training3/config.json
I0625 20:42:07.456245 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:23.333124 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training4/config.json
I0625 20:42:23.333827 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:39.393156 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/config.json
I0625 20:42:39.393832 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:55.296054 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training1/config.json
I0625 20:42:55.296774 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:11.231738 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training2/config.json
I0625 20:43:11.232401 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:27.290642 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training3/config.json
I0625 20:43:27.291420 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:43.246163 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training4/config.json
I0625 20:43:43.246868 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:59.186888 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/config.json
I0625 20:43:59.187621 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

In [30]:
files = listdir('./final_results/dev/')
roberta_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./final_results/dev/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_cased_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncased') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [31]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [35]:
P = 3.5
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [36]:
df_dev_ori['categories'] = total_predict
df_dev_ori.head()
df_dev_ori.to_json('./results/dev.json', orient='records', lines=True)